# Kefir community simulation using SteadierCom

In this tutorial you will learn you use a community simulation tool to analyse
metabolic cross-feeding interactions in a microbial community.

Our case-study will be the kefir community from [Blasche et al, 2021](https://www.nature.com/articles/s41564-020-00816-5).

We will try to find potential cross-feeding interactions between the four most abundant species during milk fermentation:

- Lactobacillus kefiranofaciens
- Leuconocost mesenteroides
- Lactococcus lactis
- Acetobacter fabarum

![fig1bd](images/fig1bd.png)

We will use a new simulation tool called [SteadierCom](https://github.com/cdanielmachado/steadiercom) (still under development), that we discussed during the theoretical part of this course. 

## 1. Running SteadierCom

SteadierCom is a command-line tool. We can use command-line tools in Jupyter (as if we were typing directly on a terminal) by starting our code with `!`.

Let's start with testing the help menu:

In [1]:
! steadiercom -h

usage: steadiercom [-h] [-c COMMUNITIES.TSV] [-o OUTPUT] [-m MEDIA]
                   [--mediadb MEDIADB] [--growth GROWTH] [--sample SAMPLE]
                   [--we WE] [--wr WR] [--target TARGET] [--solver SOLVER]
                   [--unlimited UNLIMITED]
                   MODELS [MODELS ...]

Simulate microbial communities with SteadierCom

positional arguments:
  MODELS                
                        Multiple single-species models (one or more files).
                        
                        You can use wild-cards, for example: models/*.xml, and optionally protect with quotes to avoid automatic bash
                        expansion (this will be faster for long lists): "models/*.xml". 

options:
  -h, --help            show this help message and exit
  -c COMMUNITIES.TSV, --communities COMMUNITIES.TSV
                        
                        Run SteadierCom for multiple (sub)communities.
                        The communities must be specified in a ta

**SteadierCom** will require several inputs:

- Genome-scale metabolic models for each species
- Relative abundances of species (optional)
- Medium composition
- Community growth rate (optional)
- "Unlimited" compounds (like water, minerals, trace metals) to exclude from cross-feeding analysis (optional)
- Sample size (for sampling the solution space)

All these data has been prepared in the expected format (you can inspect it in the *data* folder). The growth rate and relative abundances have been grossly estimated from the figure above (assuming 25-fold total biomass increase in 90 hours).

We are now ready to run (this can take a few minutes):

In [2]:
! steadiercom models/*.xml -c data/abundance.tsv -m MILK --mediadb data/milk_composition.tsv --growth 0.05 --unlimited data/unlimited.txt --sample 100

simulating kefir in MILK medium


(When the simulation is finished you should see a new file called `output.tsv`.)

## 2. Analysing results

The output of **SteadierCom** is a table in TSV format that you can inspect with a text editor or open in a spreadsheet software.

In this tutorial, we will use the powerful [**pandas**](https://pandas.pydata.org/) library. If you never used **pandas** before, please take a few minutes to get familiar with the [documentation](https://pandas.pydata.org/docs/user_guide/index.html).

Let's start by loading and inspecting the file:

In [13]:
import pandas as pd

df = pd.read_csv('output.tsv', sep='\t')

df.head(20) # show the first 20 lines

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
0,environment,L_lactis,M_lcts_e,0.189620,0.553966,1.00,kefir,MILK
1,L_lactis,environment,M_gal_e,0.140481,0.779777,0.03,kefir,MILK
2,L_kefiranofaciens,environment,M_ac_e,0.097696,1.654640,1.00,kefir,MILK
3,L_lactis,environment,M_co2_e,0.068394,1.554082,1.00,kefir,MILK
4,L_lactis,L_kefiranofaciens,M_acald_e,0.062510,1.419001,1.00,kefir,MILK
5,environment,L_mesenteroides,M_lcts_e,0.060953,0.178071,0.94,kefir,MILK
6,L_lactis,L_mesenteroides,M_glyc_e,0.043741,0.474966,0.25,kefir,MILK
7,L_lactis,environment,M_12ppd__S_e,0.039988,0.525508,0.82,kefir,MILK
8,L_mesenteroides,environment,M_co2_e,0.038076,0.865176,1.00,kefir,MILK
9,environment,L_kefiranofaciens,M_lcts_e,0.036514,0.106675,1.00,kefir,MILK


We can use the `.query()` and `.sort()` methods to find and sort entries.

Let's check who is consuming compounds from the environment, and sort them by consumption rate.

In [14]:
df.query('donor == "environment"').sort_values('mass_rate', ascending=False)

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
0,environment,L_lactis,M_lcts_e,1.896201e-01,0.553966,1.00,kefir,MILK
5,environment,L_mesenteroides,M_lcts_e,6.095304e-02,0.178071,0.94,kefir,MILK
9,environment,L_kefiranofaciens,M_lcts_e,3.651450e-02,0.106675,1.00,kefir,MILK
10,environment,L_kefiranofaciens,M_o2_e,2.754812e-02,0.860911,1.00,kefir,MILK
16,environment,L_kefiranofaciens,M_orn_e,1.301007e-02,0.097696,1.00,kefir,MILK
...,...,...,...,...,...,...,...,...
179,environment,L_mesenteroides,M_ribflv_e,1.687570e-06,0.000004,1.00,kefir,MILK
180,environment,L_kefiranofaciens,M_thm_e,1.479348e-06,0.000006,1.00,kefir,MILK
181,environment,L_lactis,M_thm_e,7.396740e-07,0.000003,1.00,kefir,MILK
182,environment,A_fabarum,M_fol_e,7.250664e-07,0.000002,1.00,kefir,MILK


### 2.1 - Inspecting interactions between species 

Let's first check for the main interactions highlighted in Fig. 5e the paper. 

> This time you will try by yourself! 🙂

![fig5e1](images/fig5e1.png)

*L. kefiranofaciens* "gives" amino acids to *L. mesenteroides* (although this is due to extracellular proteolytic activity and not "direct" cross-feeding). 

Can you select all cross-feeding interactions from *L. kefiranofaciens* to *L. mesenteroides* with a frequency of at least 0.1 ? 

In [11]:
# type your code here...

Or click below to reveal the solution:

In [15]:

df.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium


Since these simulations are based on random sampling of the solution space, 
it is possible that your results are different from the teacher or from your colleagues.

But, most likely, your result was an empty table. 

Now let's inspect what is possibly donated from *L. mesenteroides* to *L. kefiranofaciens*:

In [16]:
# type your code here...

Or click below to reveal the solution:

In [17]:

df.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and frequency > 0.1' )

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
14,L_mesenteroides,L_kefiranofaciens,M_acald_e,0.018770,0.426092,0.35,kefir,MILK
121,L_mesenteroides,L_kefiranofaciens,M_h2s_e,0.000210,0.006170,1.00,kefir,MILK
152,L_mesenteroides,L_kefiranofaciens,M_nh4_e,0.000039,0.002174,0.22,kefir,MILK


L. mesenteroides is expected to donate lactate to L. kefiranofaciens. In the simulations we observe exchange of acetaldehyde instead (this is likely due to a common issue with the models generated with CarveMe, they often ferment acetaldehyde instead of acetate and lactate).

Now let's look at the interactions between *L. lactis* and *A. fabarum*:

![fig5e2](images/fig5e2.png)

Can you inspect what is donated from *L. lactis* to *A. fabarum* ? 

In [ ]:
# type your code here... 

Or click below to reveal the solution:

In [19]:

df.query('donor == "L_lactis" and receiver == "A_fabarum" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
30,L_lactis,A_fabarum,M_4abut_e,0.004752,0.046083,0.33,kefir,MILK
48,L_lactis,A_fabarum,M_glyc_e,0.001766,0.019172,1.00,kefir,MILK
131,L_lactis,A_fabarum,M_arg__L_e,0.000127,0.000722,0.79,kefir,MILK
138,L_lactis,A_fabarum,M_anhgm_e,0.000109,0.000228,0.12,kefir,MILK


L. lactis is expected to donate lactate, GABA and some amino acids to A. fabarum. In the simulations we observe the exchange of GABA (M_4abut_e). We also observe exchange of glycerol and arginine, although these were not observed experimentally.

No exchange was observed experimentally from *A. fabarum* to *L. lactis*, but lets see if some are predicted in the simulations:

In [20]:
# type your code here... 

In [ ]:
Or click below to reveal the solution:

In [21]:

df.query('donor == "A_fabarum" and receiver == "L_lactis" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium


## 2.2 Building an interaction network

Now we will use the [**pyvis**](https://pyvis.readthedocs.io/en/latest/) library to display a network with all the cross-feeding interactions.

Actually, let's start by selecting only:

- interactions that have a frequency above 10% and a mass rate below 0.001 g/gDW/h
- interactions between species (not from/to the environment)


In [22]:
# type your code here...

# selected = df.query("...")

Or click below to reveal the solution:

In [23]:

selected = df.query("frequency > 0.1 and mass_rate > 0.001 and donor != 'environment' and receiver != 'environment'")

Building the network is simple, but a bit too much for this tutorial, so we have *"cooked it up"* for you:

In [26]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['mass_rate'])
    net.add_edge(row['compound'], row['receiver'], value=row['mass_rate'])

net.show('network.html')

network.html


You can try dragging around the nodes in the network for better visualization. 

> If the compound identifiers are hard to interpret, you can look them up in the [BiGG database](http://bigg.ucsd.edu/).

---------

Congratulations, you reached the end of the tutorial. 🥳

If you finished early, feel free to ask questions or try to help another colleague... 🙂